In [ ]:
import pandas as pd
import numpy as np
from typing import Dict, Any

# --- 定数 ---
# 最高スコアの分位層を指定（例: 5分位なら5が最も良い層）
TARGET_QUINTILE = 5
# 月次データであることを前提とするため、年間取引月数を12に設定
TRADING_MONTHS_YEAR = 12
# スコア列のデフォルト名。実際のデータに合わせて変更可能です。
DEFAULT_SCORE_COL = 'Factor_Score'

def simulate_quintile_data(score_col: str = DEFAULT_SCORE_COL) -> pd.DataFrame:
    """
    分位ポートフォリオ分析に必要な構造を持つ月次シミュレーションデータを作成します。
    ユーザーは実際のデータに置き換える必要があります。

    Args:
        score_col: スコア列として使用する名前。
    """
    print("--- ポートフォリオ分析用月次シミュレーションデータを生成中 ---")
    np.random.seed(42)

    # データを月次にする (freq='M')
    dates = pd.date_range(start='2020-01-31', periods=TRADING_MONTHS_YEAR * 5, freq='M') # 5年間のデータ
    tickers = [f'STOCK_{i:02d}' for i in range(50)] # 50銘柄

    all_data = []

    # 銘柄ごとの初期価格とスコアのベースを設定
    for ticker in tickers:
        # 価格シミュレーション: ランダムウォーク + トレンド
        initial_price = np.random.uniform(50, 200)
        # 月次リターンシミュレーション
        monthly_returns = np.random.normal(0.01, 0.05, len(dates)).cumsum()
        prices = initial_price * np.exp(monthly_returns)

        # スコアシミュレーション: 銘柄の品質（スコア）のベース
        base_score = np.random.uniform(0.1, 0.9)
        # スコアは日付によってわずかに変動すると仮定
        scores = base_score + np.random.normal(0, 0.05, len(dates))

        df_ticker = pd.DataFrame({
            'Date': dates,
            'Ticker': ticker,
            'Close': prices,
            score_col: scores # スコア列名を柔軟に設定
        })
        all_data.append(df_ticker)

    df = pd.concat(all_data).reset_index(drop=True)

    # 各日付のスコアに基づき、分位（Quintile）を計算
    def calculate_quintile(group):
        # スコアが高い方が良い層（ここではTARGET_QUINTILE）になるようにランク付け
        # スコア列名を使用
        if len(group[score_col].unique()) > TARGET_QUINTILE:
             group['Quintile'] = pd.qcut(
                group[score_col],
                q=TARGET_QUINTILE,
                labels=False,
                duplicates='drop'
            ) + 1 # 1からTARGET_QUINTILEのラベルを付ける
        else:
             # スコアが一意でない、または数が少ない場合のフォールバック
             group['Quintile'] = TARGET_QUINTILE // 2 + 1
        return group

    # 実際のデータではこのステップは完了済みと想定されますが、シミュレーションでは実行
    df = df.groupby('Date', group_keys=False).apply(calculate_quintile)

    return df


def calculate_portfolio_metrics(df: pd.DataFrame, target_quintile: int, score_col: str = DEFAULT_SCORE_COL) -> pd.DataFrame:
    """
    指定された分位層に属する銘柄のみでポートフォリオを構築し、主要なメトリクスを計算します。

    Args:
        df: 銘柄データ（'Date', 'Ticker', 'Close', 'Quintile'列を含む）
        target_quintile: 評価対象とする分位層の番号（例: 5）
        score_col: スコア列の名前 (確認用として引数に残すが、Quintile計算済みのDataFrameが前提)

    Returns:
        pd.DataFrame: ポートフォリオの累積リターン、リスク、リターン/リスクを含むDataFrame
    """

    if 'Quintile' not in df.columns:
        print(f"致命的なエラー: 分位列 'Quintile' がDataFrameに見つかりません。事前に分位計算が必要です。")
        return pd.DataFrame()

    print(f"\n--- 分位 {target_quintile} のポートフォリオを計算中 (月次データ) ---")

    # 1. 最高分位の銘柄のみを抽出
    top_quantile_df = df[df['Quintile'] == target_quintile].copy()

    if top_quantile_df.empty:
        print(f"警告: 分位 {target_quintile} に該当するデータがありません。")
        return pd.DataFrame()

    # 2. ポートフォリオ月次リターンを計算
    # 終値から月次リターンを計算
    top_quantile_df['Monthly_Return'] = top_quantile_df.groupby('Ticker')['Close'].pct_change()

    # NaNを0に置換（初月のリターンは0とする）
    top_quantile_df['Monthly_Return'] = top_quantile_df['Monthly_Return'].fillna(0)

    # 日付ごとに等ウェイト（Equal Weight）でポートフォリオの月次リターンを計算
    # 各日付で選択された全銘柄のリターンの平均がポートフォリオリターン
    portfolio_monthly_returns = top_quantile_df.groupby('Date')['Monthly_Return'].mean()

    # 最初の月のリターンは通常0かNaNになるため、0に設定
    portfolio_monthly_returns.iloc[0] = 0

    # 3. 各種メトリクスを計算 (年率換算には TRADING_MONTHS_YEAR=12 を使用)

    # 累和リターン (Summed Return)
    # 月次リターンの合計（単純リターン）
    summed_return = portfolio_monthly_returns.sum()

    # 累積リターン (Cumulative Return)
    # 複利効果を考慮した最終リターン
    cumulative_return = (1 + portfolio_monthly_returns).prod() - 1

    # リスク (Risk: Standard Deviation) - 年率換算
    # 標準偏差
    annualized_std_dev = portfolio_monthly_returns.std() * np.sqrt(TRADING_MONTHS_YEAR)

    # リターン/リスク (Return/Risk Ratio) - 年率平均リターンを使用
    # 年率平均リターン
    annualized_avg_return = portfolio_monthly_returns.mean() * TRADING_MONTHS_YEAR

    # リターン/リスク比 (シャープレシオの分子とほぼ同じ, リスクフリーレートは0と仮定)
    return_to_risk_ratio = annualized_avg_return / annualized_std_dev if annualized_std_dev != 0 else np.nan

    # 4. 結果をDataFrameに格納
    results_data = {
        'メトリクス': [
            '累積リターン (Cumulative Return)',
            '累和リターン (Summed Return)',
            'リスク (年率標準偏差)',
            '年率平均リターン',
            'リターン/リスク比'
        ],
        '値': [
            cumulative_return,
            summed_return,
            annualized_std_dev,
            annualized_avg_return,
            return_to_risk_ratio
        ]
    }

    results_df = pd.DataFrame(results_data)
    results_df['値'] = results_df['値'].map('{:.4f}'.format) # 表示を見やすくするために整形

    return results_df


# --- スクリプトの実行 ---
if __name__ == "__main__":
    # 実際のデータに合わせて、ここで使用したいスコア列の名前を指定してください。
    SCORE_COLUMN_TO_USE = DEFAULT_SCORE_COL

    # 1. 分位データフレームのシミュレーション（実データを使用する場合はこの行をコメントアウトし、
    #    代わりにユーザーのDataFrameを `quintile_df` にロードしてください）
    quintile_df = simulate_quintile_data(score_col=SCORE_COLUMN_TO_USE)

    # 2. ポートフォリオメトリクスの計算と表示
    if not quintile_df.empty:
        analysis_results = calculate_portfolio_metrics(quintile_df, TARGET_QUINTILE, score_col=SCORE_COLUMN_TO_USE)

        print("\n=======================================================")
        print(f"分位ポートフォリオ（最高層 {TARGET_QUINTILE}）分析結果")
        print("=======================================================")
        print(analysis_results.to_markdown(index=False)) # Markdown形式で表を出力
        print("=======================================================")

    else:
        print("\nエラー: ポートフォリオ計算に必要なデータがありませんでした。")